# Utility

Напишем несколько utility-классов, которые помогут нам в дальнейшем.

Все они будут находиться в `utils.py`, однако здесь я приведу их краткое описание:

1) Классы из ноутбука `00-preprocess`.
2) Класс Pipeline, который представляет из себя последовательность шагов.
3) Класс GridSearch, который позволяет инжектить параметры в подлежащий пайплайн чтобы находить оптимальную комбинацию гиперпараметров.

А ещё мы переведём все с датафреймов на `np.ndarray`, чтобы быть эффективнее внутри реализаций.

Поймём как мы организуем работу нашего пайплайна: сам полностью собранный предиктор будет предсказывать именно класс (то есть мы выбираем `argmax` вектору и проставляем такую категорию объекта). Для этого сделаем дополнительный шаг пайплайна. One-hot кодировать же мы будем данные конкретно при создании предиктора.

## Test

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns; sns.set_style()

import utils

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
X = pd.read_csv('data/processed/features.csv').to_numpy()
y = pd.read_csv('data/processed/labels.csv')['class'].to_numpy()

Создадим глупый предиктор:

In [3]:
from __future__ import annotations
from collections import Counter

class DummyPredictor(utils.BasicLayer):
    def __init__(self, clever: bool = False):
        super().__init__()
        self.clever = clever
        
    def fit(self, X: np.ndarray, y: np.ndarray) -> DummyPredictor:
        self.prediction = Counter(y).most_common(1)[0][0] if self.clever else y[0]
        return self
        
    def transform(self, X: np.ndarray) -> np.ndarray:
        return np.array([self.prediction] * X.shape[0])

In [4]:
pipeline = utils.Pipeline([
    ('dummy', DummyPredictor())
])

In [5]:
grids = utils.GridSearch(pipeline, {
    ('dummy', 'clever'): [True, False]
})

In [6]:
grids.fit(X, y).history

2it [00:00, 96.14it/s]


[((True,), 0.18135376756066413), ((False,), 0.16596343178621661)]

Как можно видеть, наш гридсерч с dummy классификатором работает (хоть и очень плохо).